# Cloudant, GeoJson, Geohashing, and Folium 

It displays the signal strengths in a graded color on a map. 
It uses data from an iOS app called RF Checker. This creates a .csv file of the signal strength of the RF antennae as you walk about
This version assumes all the data points have been loaded into a cloudant database
Doc on 
- [cloudant-python library](https://python-cloudant.readthedocs.io/en/latest/index.html)
- [Pandas library](https://pandas.pydata.org/pandas-docs/stable/)
- [geojson](https://pypi.python.org/pypi/geojson/2.3.0#feature)

First we install some libraries
The beginnings of this notebook came from [an Article by George T. Silva](https://georgetsilva.github.io/posts/mapping-points-with-folium/)

[Folium](http://python-visualization.github.io/folium/docs-master/index.html) is an interface to the [leaflet.org](http://leaflet.org) (also see [leafletjs](http://leafletjs.com/examples.html)) mapping API. It is used to help plot point on a map.

[Branca](https://github.com/python-visualization/branca/tree/master/branca) is also a part of Folium

Import or install libraries uses in notebook

In [1]:
import sys
import time
import pandas as pd

# import Cloudant Library. Install if necessary
try:
    import cloudant
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install --pre cloudant 
    else:
        !pip install --user --pre cloudant
    import cloudant

# import geoJson Library. Install if necessary
try:
    import geojson
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install --pre geojson 
    else:
        !pip install --user --pre geojson
    import geojson

# import geohash Library. Install if necessary
try:
    import geohash2 as Geohash
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install --pre geohash2 
    else:
        !pip install --user --pre geohash2
    import geohash2 as Geohash

# import a second python-geohash Library,. Install if necessary
try:
    Geohash2 = __import__("geohash")
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install --pre python_geohash 
    else:
        !pip install --user --pre python_geohash
    Geohash2 = __import__("geohash")

# import folium mapping Library. Install if necessary
try:
    import folium
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install folium 
    else:
        !pip install --user folium
    import folium

Credentials to access the GeoJSON database in Cloudant

In [2]:
!pip list

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
alabaster (0.7.10)
anaconda-client (1.6.5)
anaconda-project (0.8.0)
asn1crypto (0.22.0)
astroid (1.5.3)
astropy (2.0.2)
astunparse (1.5.0)
Babel (2.5.0)
backports.shutil-get-terminal-size (1.0.0)
backports.weakref (1.0rc1)
beautifulsoup4 (4.6.0)
biopython (1.69)
bitarray (0.8.1)
bkcharts (0.2)
blaze (0.11.3)
bleach (2.0.0)
bokeh (0.12.10)
boto (2.48.0)
boto3 (1.4.7)
botocore (1.7.20)
Bottleneck (1.2.1)
branca (0.3.1)
brunel (2.3)
certifi (2018.10.15)
cffi (1.10.0)
chardet (3.0.4)
click (6.7)
cloudant (2.10.2)
cloudpickle (0.4.0)
clyent (1.2.2)
colorama (0.3.9)
colour (0.1.5)
contextlib2 (0.5.5)
cplex (12.8.0.1)
cryptography (2.2.2)
cycler (0.10.0)
Cython (0.26.1)
cytoolz (0.8.2)
dask (0.15.4)
datashape (0.5.4)
debtcollector (1.17.0)
decorator (4.1.2)
dill (0.2.7.1)
distributed

In [3]:
# The code was removed by Watson Studio for sharing.

Some utility functions for later

In [4]:
# Design document used in Cloudant for all indexes
ddoc_name = 'geosignal'

# remove the annoying 'doc' tag from the rows
def removeDocTag(queryResult):
    if len(queryResult['rows']) > 0 and 'doc' in queryResult['rows'][0]:
        queryResult['rows'] = [item['doc'] for item in queryResult['rows']]
    return queryResult
        
# Retrieves documents based on a view and a key
def getViewResults(db, index, key, limit):
    if len(key) > 1:
        queryResult = db.get_view_result(ddoc_name, index, keys=keys, group=True, include_docs=True, limit=limit, raw_result=True)
    else:
        queryResult = db.get_view_result(ddoc_name, index, key=keys[0], include_docs=True, limit=limit, raw_result=True)
    return removeDocTag(queryResult)

# Retrieves documents based on an index and a query
def getSearchResults(db, index, query, bookmark, limit):
    if bookmark == '':
        queryResult = db.get_search_result(ddoc_name, index, query=query, include_docs=True, limit=limit)
    else:
        queryResult = db.get_search_result(ddoc_name, index, query=query, include_docs=True, limit=limit, bookmark=bookmark)
    return removeDocTag(queryResult)

# pauses the thread. This is useful to keep the queroes/second < 5 to Cloudant
# The caller needs to send thew appropriate value of NumCalls
def pauseCalls(numCalls):
    # The free version of cloudant limits the number of call/sec to 5
    numCalls = numCalls + 1
    if numCalls % 4 == 0:
        time.sleep(2)
    return numCalls

At this point we want to extract only a minimal # of coordinates based on a chess board, using geohash markers as a way to proceed

We want to create markers for each geohash group

+ find a geohash in the data for a specific {lat, long}
+ find the number of points in that geohash
+ print that on a map

if we use a bounding box using 7 digit geohash we get a 153 meter square of data points ==> for my data that is geohash 9xj3gej


In [5]:
mygeohash= '9xj3gdy'
mygeohash= '9xj3gdyy'
#mygeohash= '9xj3gejwz'

geohashCenter = Geohash.decode_exactly(mygeohash)
# decode_exactly gives us 4 values. We need the first 2
centerOfMap = [ x for x in geohashCenter[:2] ] 
centerOfMap

[39.69883918762207, -104.95393753051758]

Useful GeoJson Utility functions

In [6]:
import statistics as Math

def geoJsonFeature(geojsonStruct, properties):      
    feature = geojson.Feature(geometry=geojsonStruct, properties=properties)
    feature['id'] = properties['geohash']
    return feature

def getProperties(geojsonrows, key):
    if len(rows) > 0:
        signalStrengths = [ x['properties']['signalStrength'] for x in geojsonrows ]
        signalStrengthMean = Math.mean(signalStrengths)
        count = len(geojsonrows)
        # get a list of signal strengths
        strengths = []
        for x in set(signalStrengths):
            strengths.append({ 
                    'signalStrength': x, 
                    'count': signalStrengths.count(x)
            })
    else:
        signalStrengthMean = 0
        strengths = [{'signalStrength': 0,'count': 0}]
        count = 0
    return {
        'signalStrengths': strengths,
        'signalStrengthMean': signalStrengthMean,
        'geohash': key,
        'count': count
    }
           
def convertToGeoJsonMultiPoint(geojsonRows,key):
    return geoJsonFeature(
            geojson.MultiPoint([ item['geometry']['coordinates'] for item in geojsonRows ]),
            properties=getProperties(rows,key))
    
def convertToGeoJsonPolygon(geojsonRows, key):
    feature = geoJsonFeature(
            geojson.Polygon([[ item['geometry']['coordinates'] for item in geojsonRows ]]), 
            properties=getProperties(geojsonRows,key))

def bboxAsLatLong(bbox):
    rectangle = []
    for x in [['w', 'n'], ['e', 'n'], ['e', 's'], ['w', 's']]:
        rectangle.append([bbox[x[0]], bbox[x[1]]])
    return rectangle 

def convertToGeoJsonRectangle(geojsonRows, key):
    geohashbox = Geohash2.bbox(key)
    return geoJsonFeature(
            geojson.Polygon([bboxAsLatLong(geohashbox)]),
            properties=getProperties(geojsonRows,key))

def _neighbors(nearestNeighbors):
    allNeighbors = set()
    for neighbor in nearestNeighbors:
        nearestNeighbors2 = set(Geohash2.neighbors(neighbor))
        nearestNeighbors2 = nearestNeighbors2 - allNeighbors
        allNeighbors = allNeighbors | nearestNeighbors2
    return allNeighbors

def getGeohashNeighbors(geohashToSearchFor,numBoxes):
    # uses sets to manage duplicate geohashes
    allNeighbors = set([geohashToSearchFor])   
    nearestNeighbors = allNeighbors
    count = 0
    for x in range(0,numBoxes):
        nearestNeighbors = _neighbors(nearestNeighbors)
        nearestNeighbors = nearestNeighbors - allNeighbors
        allNeighbors = allNeighbors | nearestNeighbors
    return list(allNeighbors)

Search for all doucments that have a specfic set of geohashes.

In [7]:
# uses Cloudant Queries
geoJsonFeatures = []
with cloudant.cloudant_bluemix(rf_credentials,'Cloudant NoSQL DB') as client:
    db = client[db_name]
    # Run a query
    index = 'geohashView'
    limit = 200
    numCalls = 0
    keys = getGeohashNeighbors(mygeohash,5)
    bookmark = ''
    for key in keys:
        #print(len(keys))
        selector = {"geohash": { "$elemMatch": { "$eq": key} } }
        # AT some point need to use bookmarks to retreieve all values
        query = cloudant.query.Query(db,selector=selector, limit=limit)
        rows = query.result[0:limit]
        geoJsonFeatures.append(convertToGeoJsonRectangle(rows,key))
        # print(numCalls)
        numCalls = pauseCalls(numCalls)
filteredGeoJsonFeatures = [item for item in geoJsonFeatures if item['properties']['signalStrengthMean'] < 0]
len(filteredGeoJsonFeatures)

52

In [8]:
import branca.colormap as cm
# find lowest signal Strength+-**-
signalStrengthList = ([ item['properties']['signalStrengthMean'] for item in filteredGeoJsonFeatures]) if len(geoJsonFeatures) > 0 else [0]
low = min(signalStrengthList)
# find highest signal Strength
high = max(signalStrengthList)
colorScale = cm.linear.PuRd_09.scale(low, high)
colorScale.caption='Signal Strength'
colorScale

Set up Pandas DataFrame for color for geohashes

In [9]:
signalStrengths = pd.DataFrame({
    "Geohash": [ item['properties']['geohash'] for item in filteredGeoJsonFeatures ],
    "Signal Strengths": [ item['properties']['signalStrengthMean'] for item in filteredGeoJsonFeatures ]
    })
signalStrengths

,Geohash,Signal Strengths
0,9xj3gdyt,-97.724138
1,9xj3genf,-108.800000
2,9xj3gdvw,-119.000000
3,9xj3geng,-111.000000
4,9xj3gdy4,-112.000000
5,9xj3gdzq,-109.400000
6,9xj3gdyy,-100.759259
7,9xj3gdz4,-111.000000
8,9xj3gdze,-119.000000
9,9xj3gdve,-112.000000


In [10]:
signalStrengths_dict= signalStrengths.set_index('Geohash')['Signal Strengths']
signalStrengths_dict

Geohash
9xj3gdyt    -97.724138
9xj3genf   -108.800000
9xj3gdvw   -119.000000
9xj3geng   -111.000000
9xj3gdy4   -112.000000
9xj3gdzq   -109.400000
9xj3gdyy   -100.759259
9xj3gdz4   -111.000000
9xj3gdze   -119.000000
9xj3gdve   -112.000000
9xj3gen1   -111.800000
9xj3gdzm   -112.600000
9xj3gdvz   -106.142857
9xj3gdz1   -109.400000
9xj3gdyw    -96.830508
9xj3gen8   -122.000000
9xj3gene   -108.743590
9xj3gdy5   -108.333333
9xj3gdz7   -109.400000
9xj3gen9   -107.000000
9xj3gdyx   -113.595628
9xj3gej9    -98.000000
9xj3gdvx   -107.750000
9xj3gdz6   -112.600000
9xj3gdzs   -112.600000
9xj3gdvt   -119.000000
9xj3genb   -107.705882
9xj3gdym    -98.833333
9xj3gdzn   -119.000000
9xj3gen6   -110.857143
9xj3gejb   -105.500000
9xj3gdvs   -109.755814
9xj3gen0   -106.166667
9xj3gep0   -111.000000
9xj3gdvg   -111.240000
9xj3gdzp   -108.818182
9xj3gen7   -106.055556
9xj3gdyv    -99.607143
9xj3gdy1   -110.600000
9xj3gdyh   -107.000000
9xj3gdvf   -119.000000
9xj3gdvu   -111.000000
9xj3gdzt   -108.333333
9xj

In [11]:
# create a map, center it on one of the coordinates, and set zoom level
map = folium.Map(location=centerOfMap, zoom_start=18)
          
def style_function(feature): 
    color= 'black'
    try:
        color = colorScale(signalStrengths_dict[feature['id']])
    except:
        pass
    return{ 
        'color': 'black', 'weight':1, 'opacity': 0.2,
        'fillColor': color, 'fillOpactity':0.2
    }

shape = geojson.FeatureCollection(geoJsonFeatures)
folium.GeoJson(shape, name='Signal Strengths', style_function=style_function).add_to(map)
colorScale.add_to(map)
folium.LayerControl().add_to(map)
map